In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [5]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [6]:
data = requests.get(team_urls[0])

In [7]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [8]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [9]:
data = requests.get(f"https://fbref.com{links[0]}")

In [10]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [11]:
shooting.columns = shooting.columns.droplevel()

In [12]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [13]:
years = list(range(2023, 2021, -1))
all_matches = []

In [14]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [15]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

Apparently the server couldn't handle that many request in that short period of time, so i have tried

`time.sleep(10)` instead of `time.sleep(1)`

In [16]:
len(all_matches)

40

In [17]:
match_df = pd.concat(all_matches)

In [18]:
match_df.columns = [c.lower() for c in match_df.columns]

In [2]:
match_df

NameError: name 'match_df' is not defined

In [20]:
match_df.to_csv("matchess.csv")